# Detection in pump-probe geometry.
This notebook demonstrates the application of the 'sigmd' module to detection in pump-probe geometry.
This simpler, well established technique is also used as a tutorial.

It demonstrates:
- 2-step phase cycling
- Pump-probe scatter removal using phase-cycling.
- Pump-pump scatter removal using chopping.

In [11]:
from __future__ import division, print_function
import sympy as sy
from sig2d.signals import amplitude, e_field, signal, signals_for_order, Signal, phase_cycle
from sig2d.filters import pump_probe, filter_terms, contains_filter
from sig2d.signals import phase, norm

In [3]:
sy.init_printing() # pretty!

# Generate signal fields

Generate signals: up to third order with 3 pulses. We filter the indices using the pump probe filter, ie: keep only phase match along $k_3$ assuming $k_1$, $k_2$ are collinear and avoiding SHG, DC etc.

The result contains pump-probe, 2D, TG and linear signals.

In [5]:
sigs = sum(signals_for_order(i, 3, filters=pump_probe) for i in [1,2,3])
sigs

# 2-step phase cycling

## Generate detected signal

Create the probe field. The field sent to de detector is the sum of this probe field and the signals.

In [6]:
probe_field = e_field(3)
probe_field

In [7]:
detected = probe_field + sigs
detected = sy.expand(sy.conjugate(detected)*detected)
len(detected.args)

The intensity contains a lot of terms! Luckily, we can neglect quite a lot of them: anything that doesn't contain the probe field (local oscillator) will be very weak.

This is implemented using a term filter: the representation of each term needs to contain the string `"E_"`. This reduces the expression to 21 terms, which is much more manageable.

In [7]:
detected = filter_terms(contains_filter("E_"), detected)
len(detected.args)

## Manual phase cycling

The next cell demonstrates manual phase cycling. The `signals` module provides tools to automate this (demonstrated later), but this details the underlying process.

The 2-step phase cycling can be achieved by cycling the phase of pulse 1 $\phi_1$ throught (0, $\pi$), and taking the difference. This amounts to weighting each by factors (1, -1).


In [8]:
phases = [0, sy.pi]
# Next line follows Tan's nomenclature
factors = [sy.exp(sy.I*p) for p in phases]
phases, factors

Phase cycling is achieved using sympy's substitution, replacing $A_1$ by $e^{i\phi_1}$, and replacing $A_2$ and $A_3$ by 1.

In [9]:
cycled = sum(
    [f*detected.subs(  # we supply a list of (variable, substitute)
        [(amplitude(1), sy.exp(-sy.I*p)),
         (amplitude(2), 1),
         (amplitude(3), 1)])
     for f, p in zip(factors, phases)
    ])
cycled

This is the real part of the sum of the rephasing and non-rephasing signals, interfering with $\overline{E_3}$. A well known result.

# 4-step phase cycling: Scatter removal

## Detected signal, with scatter

Define a scatter amplitude $\alpha$. We now detect $\alpha(E_1+E_2)$.

In [8]:
alpha = sy.symbols(r"\alpha", real=True)
detected = sigs + probe_field + alpha*(e_field(1)+e_field(2))
detected = sy.expand(sy.conjugate(detected)*detected)
len(detected.args)

We will now neglect signal-signal cross terms and scatter-signal cross terms.

In [9]:
detected = filter_terms(contains_filter("E_"), detected)
detected = filter_terms(lambda s: not (r'\chi' in repr(s) and r'\alpha' in repr(s)),
                        detected)
len(detected.args)

## Two step phase cycling fails

We now showcase automatic phase cycling using `phase_cycle`. For this, we need to prepare a table of phases (for all 3 pulses) and weights.

In [10]:
# columns are pulses, rows are steps. 
phases = [[0,     0, 0],
          [sy.pi, 0, 0]]
weights = [1, -1]
phase_cycle(detected, phases, weights=weights)

The scatter signals still show up: both pump-probe scatter (ex: $\alpha E_1 \overline{E_3}$)
and pump-pump scatter (ex: $\alpha^2 E_1 \overline{E_2}$) are still present.

## 4 step phase cycling removes pump-probe scatter

In [15]:
phases = [[    0,     0, 0],
          [    0, sy.pi, 0],
          [sy.pi, sy.pi, 0],
          [sy.pi,     0, 0]]
weights = [1, -1, 1, -1]
phase_cycle(detected, phases, weights)

Only pump-pump scatter survives phase cycling. It's amplitude goes like $\alpha^2$: it should be very small for most samples.

## 8 step scatter removal: cycling + chopping

We can use python's `itertools` module to help in generating the tables.

In [22]:
from itertools import cycle, repeat, chain, product

In [47]:
phases = [[    0,     0, 0],
          [    0, sy.pi, 0],
          [sy.pi, sy.pi, 0],
          [sy.pi,     0, 0]]
# repeat each row twice
phases = list(chain(*zip(phases, phases)))
phases

Cycle these two rows indefinitely

In [48]:
amps = cycle([[1,1,1], [1,1,0]])
[amps.next() for i in range(4)]

In [49]:
weights = [1, -1, -1, 1, 1, -1, -1, 1]
phase_cycle(detected, phases, weights, amps)

All scatter is gone.